# Bachelor's Thesis Notebook

This is a notebook containing the results of the models of my bachelor's thesis.


In [1]:
import pandas as pd, numpy as np
import statistics

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


QUEL RITARDO DI DATASET LORO NON ERA SORTATO, QUINDI ANDAVO A METTER IL RANK MA NON MATCHAVA COL VERO ORDINE OBV, BASTARDI INFAMI

## Berger et al. 2020 Models

In [19]:
df = pd.read_csv("/content/drive/MyDrive/Bachelor's Thesis/Files/You_S1_Data_NoBillboardRanking.csv", encoding = "ISO-8859-1")
df.head(5)

,uniq_id,date,month,year,d2014,d2015,d2016,q1_14,q2_14,q3_14,...,way,well,were,what,when,where,who,will,with,yeah
0,22,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,28,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,40,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,41,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,44,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


The original dataset is **NOT** sorted appearently, this needs to be fixed since the response variable (ie. "*Song Rank*") is missing, and it must be created from scratch.

Thankfully, once sorted by "*uniq_id*", the 4200 songs will preserve the same ordering they had on the various ranking charts of *Billboard.com*, and thus it is enough to enumerate each block of 50 rows with numbers decreasing from 50 to 1.

The songs in fact were scraped from 84 different digital song sales charts, belonging to 7 major genres (ie. christian, country, dance, rock, pop, rap, r&b) and different time periods (4 times a year for 3 years).

In [21]:
df = df.sort_values(by=['uniq_id'])
df = df.reset_index(drop=True)
df.head(5)

,uniq_id,date,month,year,d2014,d2015,d2016,q1_14,q2_14,q3_14,...,way,well,were,what,when,where,who,will,with,yeah
0,1,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
1,2,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,3,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,4,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,5,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [22]:
#Build Song Rank Column
df["Rank"] = 0
i, j = 0, np.arange(50, 0, -1)
while i <= len(df.genre)-1:
    df.loc[i:i+49, "Rank"] = j
    i+=50

In [25]:
df["Rank"].head(5)

0    50
1    49
2    48
3    47
4    46
Name: Rank, dtype: int64

### Model 1

In [26]:
import statsmodels.formula.api as smf
results = smf.ols('Rank ~ you', data = df).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     11.21
Date:                Fri, 02 Dec 2022   Prob (F-statistic):           0.000822
Time:                        01:18:49   Log-Likelihood:                -17165.
No. Observations:                4200   AIC:                         3.433e+04
Df Residuals:                    4198   BIC:                         3.435e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.5217      0.367     66.768      0.0

In [ ]:
songTopicComp = []
for x in range(len(df.uniq_id)):
    songTopicComp.append(list(df[["lda10topic{}".format(x) for x in range(1,11)]].iloc[x]))

genres = ["Dan", "Pop", "Rap", "RnB", "Roc", "Chr", "Cou"]
dic_genres = {"Dan":0, "Pop":1, "Rap":2, "RnB":3, "Roc":4, "Chr":5, "Cou":6}
avgTopComp = []

for x in genres:
    aa = [songTopicComp[y] for y in range(len(df.genre)) if df.genre[y]==x]
    temp=[]
    for y in range(10):
        temp.append(statistics.mean([aa[z][y] for z in range(len(aa))]))
    avgTopComp.append(temp)

lsm = []
for x in range(len(df.genre)):
    lsm.append(
        statistics.mean(list(np.divide(abs(np.array(songTopicComp[x]) - np.array(avgTopComp[dic_genres[df.genre[x]]])),
                           np.array(songTopicComp[x]) + np.array(avgTopComp[dic_genres[df.genre[x]]]) + 0.0001)))
    )
df["lsm"] = lsm

In [ ]:
import statsmodels.api as sm
X = sm.add_constant(df["you"])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.618
Date:                Thu, 01 Dec 2022   Prob (F-statistic):              0.203
Time:                        00:00:54   Log-Likelihood:                -17170.
No. Observations:                4200   AIC:                         3.434e+04
Df Residuals:                    4198   BIC:                         3.436e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         25.1279      0.368     68.340      0.0

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
